# Day 17 - Chronospatial Computer part2

Determine which value gives result of itself

In [1]:
TEST = False

INPUT_FN = "input_test2.txt"
TEST_SOLUTION = 117440

if not TEST:
    INPUT_FN = "input.txt"


In [2]:
with open(INPUT_FN) as f:
    s= f.read()

    regs, prog = s.split('\n\n')

In [3]:
regs

'Register A: 28066687\nRegister B: 0\nRegister C: 0'

In [4]:
prog

'Program: 2,4,1,1,7,5,4,6,0,3,1,4,5,5,3,0\n'

In [5]:
RA,RB,RC = 0,0,0

In [6]:
prog_2 = prog.split(':')[1]
prog_ints =  [int(is0) for is0 in prog_2.split(',')]

In [7]:
prog_2= prog_2.strip()
prog_2

'2,4,1,1,7,5,4,6,0,3,1,4,5,5,3,0'

In [8]:
prog_ints

[2, 4, 1, 1, 7, 5, 4, 6, 0, 3, 1, 4, 5, 5, 3, 0]

In [9]:
instructions = []
for i in range(0,len(prog_ints),2):
    instructions.append( ( prog_ints[i], prog_ints[i+1] ) )

In [10]:
instructions

[(2, 4), (1, 1), (7, 5), (4, 6), (0, 3), (1, 4), (5, 5), (3, 0)]

In [11]:
def printd(s):
    if TEST:
        print(s)

In [12]:
opcodes_int_to_str = {0:"adv",1:"bxl",2:"bst",3:"jnz",4:"bxc", 5:"out",6:"bdv", 7:"cdv"}

Run program

In [13]:
def get_output_val(RA0):
    ip=0 # units of two bytes

    output=""

    RA,RB,RC = RA0,0,0

    while ip<len(instructions):
        
        opcode,operand = instructions[ip]

        #printd(f"{opcode=}, {operand=}")

        combo_op_val = operand # default
        match operand:
            case 4:
                combo_op_val=RA
            case 5:
                combo_op_val=RB
            case 6:
                combo_op_val=RC
            case 7:
                printd("combo_op_val=7, reserved")

        printd(f"{RA=}, {RB=}, {RC=}, {ip=}, {opcode=} {opcodes_int_to_str[opcode]}, {operand=}, {combo_op_val=}")
        match opcode:
            case 0: #adv
                RA = int( RA / (2**combo_op_val) )
            case 1: #bxl
                RB = RB ^ operand
            case 2: #bst
                RB = combo_op_val % 8
            case 3: #jnz
                if RA != 0:
                    ip = operand // 2
                    continue
            case 4: #bxc
                RB = RB ^ RC
            case 5: #out
                val = combo_op_val % 8
                output+=f"{val},"
            case 6: #bdv
                RB = int( RA / (2**combo_op_val) )
            case 7: #cdv
                RC = int( RA / (2**combo_op_val) )

        ip+=1

    res = output[:-1]
    return res

In [14]:
if TEST:
    res = get_output_val(TEST_SOLUTION)
    print(res)

    assert res == prog_2
    print("OK")

In [15]:
# RA=0

# while True:
#     res0 = get_output_val(RA)
#     printd(f"{RA=} , {res0=}, {prog=}")
#     if prog_2 == res0:
#         print("solution: ", res0, f", with {RA=}")
#         break
#     RA+=1

It takes ages. Maybe is best to try to understand program itself

# Program print

In [16]:
ip=0 # units of two bytes

while ip<len(instructions):
    print(f"{ip=}")
    opcode,operand = instructions[ip]

    combo_op_val = str(operand) # default
    match operand:
        case 4:
            combo_op_val="RA"
        case 5:
            combo_op_val="RB"
        case 6:
            combo_op_val="RC"
        case 7:
            printd("combo_op_val=7, reserved")

    #printd(f"{RA=}, {RB=}, {RC=}, {ip=}, {opcode=}, {operand=}, {combo_op_val=}")
    match opcode:
        case 0: #adv
            print(f"adv: with combo={combo_op_val}: RA = RA/(2^combo)")
            #RA = int( RA / (2**combo_op_val) )
        case 1: #bxl
            print(f"bxl: RB = RB XOR {operand=}")
            #RB = RB ^ operand
        case 2: #bst
            print(f"bst: RB = {combo_op_val} mod 8")
            #RB = combo_op_val % 8
        case 3: #jnz
            print(f"jnz: if RA!=0, jump to ip={operand//2}")
            # if RA != 0:
            #     ip = operand // 2
            #     continue
        case 4: #bxc
            print(f"bxc: RB = RB XOR RC")
            #RB = RB ^ RC
            
        case 5: #out
            print(f"OUT: {combo_op_val} % 8 ,")
            #val = combo_op_val % 8
            #output+=f"{val},"
        case 6: #bdv
            print(f"bdv: RB = RA / 2^{combo_op_val}")
            #RB = int( RA / (2**combo_op_val) )
        case 7: #cdv
            #RC = int( RA / (2**combo_op_val) )
            print(f"cdv: RC = RA / 2^{combo_op_val}")

    ip+=1

ip=0
bst: RB = RA mod 8
ip=1
bxl: RB = RB XOR operand=1
ip=2
cdv: RC = RA / 2^RB
ip=3
bxc: RB = RB XOR RC
ip=4
adv: with combo=3: RA = RA/(2^combo)
ip=5
bxl: RB = RB XOR operand=4
ip=6
OUT: RB % 8 ,
ip=7
jnz: if RA!=0, jump to ip=0


* It looks like a sort of while loop, using RA as counter.
* Each iteration reduces the calue of RA by RA = RA/(2^3) = RA/8.
* We have 8*2=16 'bytes', so we need RA to be 8^16 at least. Or considering a byte to have 3 bits, it should have 16 bytes.
* Following the algorithm

0. RB = last 3 bits of RA
1. RB1 = RB XOR 001 # some sort of encoding
2. RC = RA / 2^RB1  # RB becomes RA shifted by encoded RB times
3. RB2 = RB1 XOR RC
4. (RA only), where RA bytes are shifted. last 3 bits are removed
5. RB3 = RB2 XOR 100
6. print RB3 mod 8


But now I would like to do this in reverse to find A

Program is [2, 4, 1, 1, 7, 5, 4, 6, 0, 3, 1, 4, 5, 5, 3, 0]


In [17]:
len(instructions)

8

Try to construct RA based in this code, but going on reverse

In [21]:
RA0 = 0
v1s = [0,1,2,3]
print(f"starting {RA0=}")
for prg in prog_ints[::-1]:
    RB3 = prg ^ 0b100 # (out = v2 xor 100)
    RB3_bstr = format(RB3, "03b")
    print(f"{prg=}, out1 = {RB3_bstr}")

    RA1=None

    # Try v1s and get the first consistency
    for v1 in v1s:
        v1_dec = v1 ^ 0b001
        RC = RB3 ^ v1

        is_consistent=False
        if v1==3:
            is_consistent=True
        else:
            ndigits = 3-v1
            RC_str = format(RC, "03b")
            v1_dec_str = format(v1_dec, "03b")
            print(f"{v1=}, {RC_str=}, {v1_dec_str=}")

            if v1_dec_str[:ndigits] == RC_str[-ndigits:]:
                is_consistent=True

        if is_consistent:
            print(f"consistent with {v1=}")
            #use v1
            RA1 = (RC<<v1) | v1_dec
            break
    
    RA2 = (RA0<<3) | RA1
    print (f"new RA = {RA2} , in binary {bin(RA2)}")
    RA0=RA2


starting RA0=0
prg=0, out1 = 100
v1=0, RC_str='100', v1_dec_str='001'
v1=1, RC_str='101', v1_dec_str='000'
v1=2, RC_str='110', v1_dec_str='011'
consistent with v1=2
new RA = 27 , in binary 0b11011
prg=3, out1 = 111
v1=0, RC_str='111', v1_dec_str='001'
v1=1, RC_str='110', v1_dec_str='000'
v1=2, RC_str='101', v1_dec_str='011'
consistent with v1=3
new RA = 250 , in binary 0b11111010
prg=5, out1 = 001
v1=0, RC_str='001', v1_dec_str='001'
consistent with v1=0
new RA = 2001 , in binary 0b11111010001
prg=5, out1 = 001
v1=0, RC_str='001', v1_dec_str='001'
consistent with v1=0
new RA = 16009 , in binary 0b11111010001001
prg=4, out1 = 000
v1=0, RC_str='000', v1_dec_str='001'
v1=1, RC_str='001', v1_dec_str='000'
v1=2, RC_str='010', v1_dec_str='011'
consistent with v1=2
new RA = 128075 , in binary 0b11111010001001011
prg=1, out1 = 101
v1=0, RC_str='101', v1_dec_str='001'
v1=1, RC_str='100', v1_dec_str='000'
consistent with v1=1
new RA = 1024600 , in binary 0b11111010001001011000
prg=3, out1 = 111


In [22]:
TEST=True
get_output_val(27)

RA=27, RB=0, RC=0, ip=0, opcode=2 bst, operand=4, combo_op_val=27
RA=27, RB=3, RC=0, ip=1, opcode=1 bxl, operand=1, combo_op_val=1
RA=27, RB=2, RC=0, ip=2, opcode=7 cdv, operand=5, combo_op_val=2
RA=27, RB=2, RC=6, ip=3, opcode=4 bxc, operand=6, combo_op_val=6
RA=27, RB=4, RC=6, ip=4, opcode=0 adv, operand=3, combo_op_val=3
RA=3, RB=4, RC=6, ip=5, opcode=1 bxl, operand=4, combo_op_val=3
RA=3, RB=0, RC=6, ip=6, opcode=5 out, operand=5, combo_op_val=0
RA=3, RB=0, RC=6, ip=7, opcode=3 jnz, operand=0, combo_op_val=0
RA=3, RB=0, RC=6, ip=0, opcode=2 bst, operand=4, combo_op_val=3
RA=3, RB=3, RC=6, ip=1, opcode=1 bxl, operand=1, combo_op_val=1
RA=3, RB=2, RC=6, ip=2, opcode=7 cdv, operand=5, combo_op_val=2
RA=3, RB=2, RC=0, ip=3, opcode=4 bxc, operand=6, combo_op_val=0
RA=3, RB=2, RC=0, ip=4, opcode=0 adv, operand=3, combo_op_val=3
RA=0, RB=2, RC=0, ip=5, opcode=1 bxl, operand=4, combo_op_val=0
RA=0, RB=6, RC=0, ip=6, opcode=5 out, operand=5, combo_op_val=6
RA=0, RB=6, RC=0, ip=7, opcode=3 j

'0,6'

In [23]:
bin(0b110 ^ 0b010)

'0b100'

In [24]:
bin(27)

'0b11011'

In [25]:
TEST=False
for i in range (64):
    print(f"{i}: {get_output_val(i)}")

0: 5
1: 5
2: 7
3: 6
4: 1
5: 0
6: 3
7: 2
8: 1,5
9: 5,5
10: 6,5
11: 4,5
12: 1,5
13: 0,5
14: 3,5
15: 2,5
16: 5,7
17: 5,7
18: 5,7
19: 2,7
20: 1,7
21: 1,7
22: 3,7
23: 2,7
24: 1,6
25: 5,6
26: 4,6
27: 0,6
28: 1,6
29: 1,6
30: 3,6
31: 2,6
32: 5,1
33: 5,1
34: 3,1
35: 6,1
36: 0,1
37: 2,1
38: 3,1
39: 2,1
40: 1,0
41: 5,0
42: 2,0
43: 4,0
44: 0,0
45: 2,0
46: 3,0
47: 2,0
48: 5,3
49: 5,3
50: 1,3
51: 2,3
52: 0,3
53: 3,3
54: 3,3
55: 2,3
56: 1,2
57: 5,2
58: 0,2
59: 0,2
60: 0,2
61: 3,2
62: 3,2
63: 2,2


Try a different algorithm, whereby register A will start with several 3bit combinations, each combination is adjusted to match respective target at the end

In [26]:
RAs = [0]*len(prog_ints)
RAs

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [27]:
len(prog_ints)

16

In [95]:
RA_input=0
for r0 in RAs:
    RA_input+= r0
    RA_input<<3
RA_input

def get_input_from_ras(ras):
    ra_res=0

    for r0 in ras:
        ra_res+= r0
        res = ra_res
        ra_res = ra_res<<3 # prepare for next
    return res

try to match response with prog_ints below. Adjust digits in list from left to tight. Value ranges is 0-7

In [ ]:
#inp_list = [5,6,1,1,3,0,  3, 7, 0, 0, 0, 0, 0, 0, 0, 1 ]
#inp_list = [5,6,1,0]
inp_list= [5, 6, 1, 1, 5, 0, 4, 4, 3, 2, 0, 2, 5, 0, 5, 2] # calculated using DFS below
print(len(inp_list))
inp0 = get_input_from_ras(inp_list)
#inp0 = get_input_from_ras([  7 ])
print(bin(inp0))
print(inp0)
out0 = get_output_val(inp0)
print(out0)
print(len(out0.split(',')))

16
0b101110001001101000100100011010000010101000101010
202972175280682
2,4,1,1,7,5,4,6,0,3,1,4,5,5,3,0
16


In [100]:
prog_ints

[2, 4, 1, 1, 7, 5, 4, 6, 0, 3, 1, 4, 5, 5, 3, 0]

In [106]:
TEST=True
print(get_output_val(7)) # 7 is only that results in output of 2
TEST=False

RA=7, RB=0, RC=0, ip=0, opcode=2 bst, operand=4, combo_op_val=7
RA=7, RB=7, RC=0, ip=1, opcode=1 bxl, operand=1, combo_op_val=1
RA=7, RB=6, RC=0, ip=2, opcode=7 cdv, operand=5, combo_op_val=6
RA=7, RB=6, RC=0, ip=3, opcode=4 bxc, operand=6, combo_op_val=0
RA=7, RB=6, RC=0, ip=4, opcode=0 adv, operand=3, combo_op_val=3
RA=0, RB=6, RC=0, ip=5, opcode=1 bxl, operand=4, combo_op_val=0
RA=0, RB=2, RC=0, ip=6, opcode=5 out, operand=5, combo_op_val=2
RA=0, RB=2, RC=0, ip=7, opcode=3 jnz, operand=0, combo_op_val=0
2


In [70]:
0b110

6

In [268]:
def dfs(cur_list, next_index):
    print(f"{next_index}: {cur_list=}")
    if next_index>= len(prog_ints):
        print("Completed successfully")
        print(cur_list)
        return True
        
    target_val = prog_ints[ len(prog_ints)-next_index-1]
    
    #Try next 3-bits
    for i in range(8):
        cur_list0=cur_list.copy()
        cur_list0.append(i)
        
        out_str = get_output_val( get_input_from_ras(cur_list0) )
        
        val0 = int( out_str.split(',')[0])

        if val0==target_val:
            print(f"{next_index}: {i} is a match")
            res = dfs( cur_list0 , next_index+1)
            if res:
                return True

In [269]:
len(prog_ints)

16

In [270]:
dfs([],0)

0: cur_list=[]
0: 5 is a match
1: cur_list=[5]
1: 6 is a match
2: cur_list=[5, 6]
2: 0 is a match
3: cur_list=[5, 6, 0]
3: 0 is a match
4: cur_list=[5, 6, 0, 0]
3: 1 is a match
4: cur_list=[5, 6, 0, 1]
4: 3 is a match
5: cur_list=[5, 6, 0, 1, 3]
5: 0 is a match
6: cur_list=[5, 6, 0, 1, 3, 0]
3: 4 is a match
4: cur_list=[5, 6, 0, 4]
2: 1 is a match
3: cur_list=[5, 6, 1]
3: 1 is a match
4: cur_list=[5, 6, 1, 1]
4: 3 is a match
5: cur_list=[5, 6, 1, 1, 3]
5: 0 is a match
6: cur_list=[5, 6, 1, 1, 3, 0]
4: 5 is a match
5: cur_list=[5, 6, 1, 1, 5]
5: 0 is a match
6: cur_list=[5, 6, 1, 1, 5, 0]
6: 4 is a match
7: cur_list=[5, 6, 1, 1, 5, 0, 4]
7: 4 is a match
8: cur_list=[5, 6, 1, 1, 5, 0, 4, 4]
8: 3 is a match
9: cur_list=[5, 6, 1, 1, 5, 0, 4, 4, 3]
9: 2 is a match
10: cur_list=[5, 6, 1, 1, 5, 0, 4, 4, 3, 2]
10: 0 is a match
11: cur_list=[5, 6, 1, 1, 5, 0, 4, 4, 3, 2, 0]
11: 2 is a match
12: cur_list=[5, 6, 1, 1, 5, 0, 4, 4, 3, 2, 0, 2]
12: 4 is a match
13: cur_list=[5, 6, 1, 1, 5, 0, 4, 4, 

True